In [ ]:
%run includes/3-network-setup.ipynb

In [ ]:

# get networks
queries = [
    f"""
    SELECT name,
    '{project}' as project,
    mtu,
    networkFirewallPolicyEnforcementOrder as fwPolicyEnforcementOrder,
    peerings,
    JSON_EXTRACT(routingConfig, '$.routingMode') as routingMode,
    subnetworks
    FROM google.compute.networks WHERE project = '{project}'
    """
    for project in compute_enabled_projects
]

networks_df = (lambda df: df[df['name'] != 'null'])(run_stackql_queries(queries))
networks_df

In [ ]:

# network peerings
networks_subset_df = networks_df[networks_df['peerings'] != 'null'][['name', 'peerings']].copy()
network_peerings_df = explode_json_list_col(networks_subset_df, 'peerings', 'peering')
network_peerings_df

In [ ]:

# subnetworks
queries = [
    f"""
    SELECT 
    name,
    SPLIT_PART(region, '/', 9) as region,
    SPLIT_PART(network, '/', 10) as network,
    stackType,
    ipCidrRange,
    secondaryIpRanges,
    gatewayAddress,
    ipv6CidrRange,
    purpose,
    enableFlowLogs,
    privateIpGoogleAccess
    FROM google.compute.subnetworks WHERE project = '{project}'
    """
    for project in list(networks_df['project'].unique())
]

subnetworks_df = run_stackql_queries(queries)
subnetworks_df